In [1]:
#import libraries
import boto3
import json
import time

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-testing"
region = 'eu-central-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [3]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
            print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [4]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
    print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
        for item in resultPage["Blocks"]:
            if item["BlockType"] == "LINE":
                policy = policy + item["Text"] + "\n"
                print ('\033[94m' + item["Text"] + '\033[0m')
    return policy

In [5]:
i = 0
keyword = "DOF"
lower_limit = 0
upper_limit = lower_limit + 2
for obj in s3.Bucket(s3BucketName).objects.all():#.filter(Prefix='El Salvador'):
    if keyword in obj.key: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

5 ** DOF - Saved_as_pdf.pdf
Started job with id: 9bf0f153c0c9f61fd8bd302d695015c30852079ddc395bb2e068cbaf841a6994
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
5/3/2021
DOF - Diario Oficial de la Federación
DOF: 15/02/2021
CONVENIO Modificatorio al Convenio de Coordinación en materia de reasignación de recursos, que celebran la Secretaría de
Bienestar y el Estado de Tabasco, para la operación del Programa Federal Sembrando Vida.
Al margen un sello con el Escudo Nacional, que dice: Estados Unidos Mexicanos. - BIENESTAR. - Secretaría de Bienestar.
CONVENIO MODIFICATORIO AL CONVENIO DE COORDINACION EN MATERIA DE REASIGNACION DE RECURSOS QUE CELEBRAN
POR UNA PARTE EL EJECUTIVO FEDERAL, POR CONDUCTO DE LA SECRETARIA DE BIENESTAR, A LA QUE EN ADELANTE SE
DENOMINARA "LA SECRETARIA", REPRESENT

In [ ]:
policy